In [1]:
import requests
import boto3
import json
import os
import sys
import argparse
import importlib
import transformers
import torch
import pathlib
import awswrangler as wr
from IPython.display import display
from sagemaker.huggingface.processing import HuggingFaceProcessor
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import FrameworkProcessor
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.session import get_execution_role


# Adding ../01_modules or ./01_modules to the system path so that we can load modules from 
# there as well
if '__file__' in globals():
    script_dir = pathlib.Path(__file__).parent.resolve()
else:
    script_dir = pathlib.Path().absolute()
modules_path_in_dev = os.path.abspath(os.path.join(script_dir, '..', '01_modules'))
modules_path_in_prod = os.path.abspath(os.path.join(script_dir, '01_modules'))
if os.path.exists(modules_path_in_dev):
    sys.path.append(modules_path_in_dev)
if os.path.exists(modules_path_in_prod):
    sys.path.append(modules_path_in_prod)


# # Jupyter only reads a local module the first time after 
# # kernel start. Re-running a cell with 
# # "from mymodulename import *" would not change
# # anything, even if the imported module has since changed.
# # As a workaround, we need to directly load the module, 
# # use importlib.reload to reload it and then import * 
import utils
_ = importlib.reload(utils)
import config
_ = importlib.reload(config) 


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
config.py loaded: v0.1
utils.py loaded: v0.2.12
utils.py loaded: v0.2.12
config.py loaded: v0.1


In [2]:
if False: # do not run this again, takes a day
    execution_role = get_execution_role()
    source_dir = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
    print('source_dir:', source_dir)
    sklearn_processor = FrameworkProcessor(
        estimator_cls=SKLearn,
        framework_version='1.2-1', # The newest supported version by sagemaker
        instance_type='ml.c7i.16xlarge',
        instance_count=1,
        base_job_name=f'openalex_works_reduction'.replace('_','-'),
        role=execution_role
    )
    
    step_args = sklearn_processor.run(
        code='src/03_transformation/03_11_transformation_openalex_works_reduction.py',
        source_dir=source_dir,
        inputs=[], # We are not using automatic input-output mapping, instead we handle everything in the script directly on S3
        outputs=[],
        arguments=[
            '--runtype', 'prod',
            '--file-max-limit', '10000',
        ],
        wait=True
    )

In [2]:
table_name = 'base_openalex_works_reduced'
timelogger = utils.TimeLogger()
utils.create_table_from_sql_file(
    database_name = '02_stg',
    table_name = table_name,
    overwrite_strategy='overwrite', # options: fail, overwrite, ignore
    wait=True,
)
timelogger.log(f'"02_stg".{table_name} created')

utils.pd_set_options(cols=500)
display(wr.athena.read_sql_query(f"""SELECT * FROM "02_stg".{table_name} LIMIT 3 """, '02_stg'))
display(wr.athena.read_sql_query(f"""SELECT COUNT(*) AS c FROM "02_stg".{table_name} """, '02_stg'))
timelogger.log(f'"02_stg".{table_name} queries finished')

 :: :: TIMELOGGER STARTED :: | since_start: 0.00 seconds | since_last: 0.00 seconds :: 
Table 02_stg.base_openalex_works_reduced already exists. Overwriting since overwrite_strategy=="overwrite".
Deleting table from Glue Catalog 02_stg base_openalex_works_reduced
Deleting S3 objects from s3://sagemaker-research-methodology-extraction/01_data/02_stg/base_openalex_works_reduced/
s3_parent_target_path:  s3://sagemaker-research-methodology-extraction/01_data/02_stg
 :: "02_stg".base_openalex_works_reduced created | since_start: 1.0 minute, 48.22 seconds | since_last: 1.0 minute, 48.22 seconds :: 


,id_openalex,id_doi,title,language,primary_topic_id,primary_topic_display_name,primary_topic_subfield_id,primary_topic_subfield_display_name,primary_topic_field_id,primary_topic_field_display_name,primary_topic_domain_id,primary_topic_domain_display_name
0,2489876428,<NA>,TLR5 rs2072493 A/G基因多态性与广西人群抗中性粒细胞胞浆抗体相关性小血管炎的关系,zh-cn,T11316,Mycobacterium research and diagnosis,2713,Epidemiology,27,Medicine,4,Health Sciences
1,2489876434,10.1007/11376316_3,Control of Nonlinear Systems with Uncertainty,en,T10046,Stability and Control of Uncertain Systems,2207,Control and Systems Engineering,22,Engineering,3,Physical Sciences
2,2489876500,10.1016/b978-1-4377-2207-9.00052-5,Peripheral Vascular Disease,en,T10926,Peripheral Artery Disease Management,2746,Surgery,27,Medicine,4,Health Sciences


,c
0,270051911


 :: "02_stg".base_openalex_works_reduced queries finished | since_start: 1.0 minute, 59.92 seconds | since_last: 11.70 seconds :: 


' :: "02_stg".base_openalex_works_reduced queries finished | since_start: 1.0 minute, 59.92 seconds | since_last: 11.70 seconds :: '

In [4]:
utils.pd_set_options()
display(wr.athena.read_sql_query("""SELECT * FROM "02_stg".base_openalex_works_reduced LIMIT 5 """, '02_stg'))
display(wr.athena.read_sql_query("""SELECT COUNT(*) AS c FROM "02_stg".base_openalex_works_reduced """, '02_stg'))

,id_openalex,id_doi,title,language,primary_topic_id,primary_topic_display_name,primary_topic_subfield_id,primary_topic_subfield_display_name,primary_topic_field_id,primary_topic_field_display_name,primary_topic_domain_id,primary_topic_domain_display_name
0,269677805,<NA>,2. Profil type du détenu politique à Eysses,fr,T10153,"Education, sociology, and vocational training",3312,Sociology and Political Science,33,Social Sciences,2,Social Sciences
1,2696780303,<NA>,Analisa Struktur Dan Material Speed Bump Dengan Bahan Concrete Foam Untuk Penggerak Tenaga Listrik,id,T13674,Computer Science and Engineering,1702,Artificial Intelligence,17,Computer Science,3,Physical Sciences
2,2696784097,<NA>,Strategies of survival during the holocaust,en,T11203,Jewish and Middle Eastern Studies,3312,Sociology and Political Science,33,Social Sciences,2,Social Sciences
3,2696788076,<NA>,Business intelligence and Marketing analytics/Inteligencia de negocio y análisis de datos,<NA>,T11891,Big Data and Business Intelligence,1404,Management Information Systems,14,"Business, Management and Accounting",2,Social Sciences
4,2696777682,<NA>,El papel transversal de la lectura en el currículo,es,T13061,Literacy and Educational Practices,3304,Education,33,Social Sciences,2,Social Sciences


,c
0,270051911


In [3]:
table_name = 'base_semanticscholar_s2orcv2'
timelogger = utils.TimeLogger()
utils.create_table_from_sql_file(
    database_name = '02_stg',
    table_name = table_name,
    overwrite_strategy='overwrite', # options: fail, overwrite, ignore
    wait=True,
)
timelogger.log(f'"02_stg".{table_name} created')

utils.pd_set_options(cols=500)
display(wr.athena.read_sql_query(f"""SELECT * FROM "02_stg".{table_name} LIMIT 3 """, '02_stg'))
display(wr.athena.read_sql_query(f"""SELECT COUNT(*) AS c FROM "02_stg".{table_name} """, '02_stg'))
timelogger.log(f'"02_stg".{table_name} queries finished')

 :: :: TIMELOGGER STARTED :: | since_start: 0.00 seconds | since_last: 0.00 seconds :: 
Table 02_stg.base_semanticscholar_s2orcv2 already exists. Overwriting since overwrite_strategy=="overwrite".
Deleting table from Glue Catalog 02_stg base_semanticscholar_s2orcv2
Deleting S3 objects from s3://sagemaker-research-methodology-extraction/01_data/02_stg/base_semanticscholar_s2orcv2/
s3_parent_target_path:  s3://sagemaker-research-methodology-extraction/01_data/02_stg
 :: "02_stg".base_semanticscholar_s2orcv2 created | since_start: 11.0 minutes, 1.71 seconds | since_last: 11.0 minutes, 1.71 seconds :: 


,id_semanticscholar,id_mag,id_doi,id_arxiv,title,source_url,openaccess_status,content_text,annotations_paragraph,annotations_section_header,license
0,1808709,2101143116,10.1186/1476-511x-13-132,<NA>,Twist 1 regulates the expression of PPARγ during hormone-induced 3T3-L1 preadipocyte differentiation: a possible role in obesity and associated diseases,https://pmc.ncbi.nlm.nih.gov/articles/PMC4150960,GOLD,"\nBackground\n\nObesity has become an epidemic in the human population, and China has the highest number of obese patients in the world [1]. Because obesity involves an increase in the number of adipocytes, any of the factors involved in adipocyte differentiation might be of great importance for the development of obesity. To date, numerous factors and proteins have been implicated in the generation of new fat cells, including peroxisome proliferator-activated receptor gamma (PPARγ) [2,3], C...","[{""attributes"":null,""end"":781,""start"":13},{""attributes"":null,""end"":2087,""start"":783},{""attributes"":null,""end"":3470,""start"":2089},{""attributes"":null,""end"":4312,""start"":3472},{""attributes"":null,""end"":5329,""start"":4323},{""attributes"":null,""end"":5818,""start"":5409},{""attributes"":null,""end"":6151,""start"":5890},{""attributes"":null,""end"":6744,""start"":6218},{""attributes"":null,""end"":8054,""start"":6870},{""attributes"":null,""end"":8511,""start"":8056},{""attributes"":null,""end"":9031,""start"":8525},{""attributes"":n...","[{""attributes"":null,""end"":11,""start"":1},{""attributes"":null,""end"":4321,""start"":4314},{""attributes"":null,""end"":5407,""start"":5331},{""attributes"":null,""end"":5888,""start"":5820},{""attributes"":null,""end"":6216,""start"":6153},{""attributes"":null,""end"":6868,""start"":6746},{""attributes"":null,""end"":8523,""start"":8513},{""attributes"":null,""end"":16548,""start"":16527},{""attributes"":null,""end"":16559,""start"":16550},{""attributes"":null,""end"":17519,""start"":17439},{""attributes"":null,""end"":19472,""start"":19445},{""attrib...",CCBY
1,3738414,2789005638,10.1155/2018/1042479,<NA>,PROM and Labour Effects on Urinary Metabolome: A Pilot Study,https://pmc.ncbi.nlm.nih.gov/articles/PMC5817378,GOLD,"\nIntroduction\n\nThe early diagnosis of pregnancy-related complications and the prediction of pregnancy outcome are considered strategic clinical goals to ensure the health of mothers and of their babies. Among these, premature rupture of membranes (PROM) consists of the rupture of the foetal membranes before the onset of labour. It can be observed at any gestational age [1] and occurs in approximately 10% of pregnant women and in roughly 40% of preterm deliveries [2]. Foetal membranes are ...","[{""attributes"":null,""end"":4623,""start"":15},{""attributes"":null,""end"":6006,""start"":4703},{""attributes"":null,""end"":7040,""start"":6040},{""attributes"":null,""end"":7808,""start"":7063},{""attributes"":null,""end"":9464,""start"":7836},{""attributes"":null,""end"":10835,""start"":9466},{""attributes"":null,""end"":11411,""start"":10861},{""attributes"":null,""end"":12575,""start"":11413},{""attributes"":null,""end"":12831,""start"":12585},{""attributes"":null,""end"":12984,""start"":12833},{""attributes"":null,""end"":13373,""start"":12986},{""...","[{""attributes"":{""n"":""1.""},""end"":13,""start"":1},{""attributes"":{""n"":""2.""},""end"":4646,""start"":4625},{""attributes"":{""n"":""2.1.""},""end"":4701,""start"":4648},{""attributes"":{""n"":""2.2.""},""end"":6038,""start"":6008},{""attributes"":{""n"":""2.3.""},""end"":7061,""start"":7042},{""attributes"":{""n"":""2.4.""},""end"":7834,""start"":7810},{""attributes"":{""n"":""3.""},""end"":10859,""start"":10837},{""attributes"":{""n"":""3.3.""},""end"":12583,""start"":12577},{""attributes"":{""n"":""4.""},""end"":13385,""start"":13375},{""attributes"":{""n"":""5.""},""end"":167...",CCBY
2,15710447,2741402565,10.18653/v1/w17-1005,<NA>,Word Embedding and Topic Modeling Enhanced Multiple Features f

,c
0,11609787


 :: "02_stg".base_semanticscholar_s2orcv2 queries finished | since_start: 11.0 minutes, 11.75 seconds | since_last: 10.04 seconds :: 


' :: "02_stg".base_semanticscholar_s2orcv2 queries finished | since_start: 11.0 minutes, 11.75 seconds | since_last: 10.04 seconds :: '

In [4]:
table_name = 'stg_semanticscholar_combined_works'
timelogger = utils.TimeLogger()
utils.create_table_from_sql_file(
    database_name = '02_stg',
    table_name = table_name,
    overwrite_strategy='overwrite', # options: fail, overwrite, ignore
    wait=True,
)
timelogger.log(f'"02_stg".{table_name} created')

utils.pd_set_options(cols=500)
display(wr.athena.read_sql_query(f"""SELECT * FROM "02_stg".{table_name} LIMIT 3 """, '02_stg'))
display(wr.athena.read_sql_query(f"""SELECT COUNT(*) AS c FROM "02_stg".{table_name} """, '02_stg'))
timelogger.log(f'"02_stg".{table_name} queries finished')

 :: :: TIMELOGGER STARTED :: | since_start: 0.00 seconds | since_last: 0.00 seconds :: 
Table 02_stg.stg_semanticscholar_combined_works already exists. Overwriting since overwrite_strategy=="overwrite".
Deleting table from Glue Catalog 02_stg stg_semanticscholar_combined_works
Deleting S3 objects from s3://sagemaker-research-methodology-extraction/01_data/02_stg/stg_semanticscholar_combined_works/
s3_parent_target_path:  s3://sagemaker-research-methodology-extraction/01_data/02_stg
 :: "02_stg".stg_semanticscholar_combined_works created | since_start: 13.0 minutes, 48.32 seconds | since_last: 13.0 minutes, 48.32 seconds :: 


,id_semanticscholar,id_mag,id_doi,id_arxiv,title,source_url,openaccess_status,content_text,annotations_paragraph,annotations_section_header,license,content_abstract,publication_year,publication_date
0,220531160,3042431448,10.1167/iovs.61.8.16,<NA>,Quantitative Fundus Autofluorescence in Rhesus Macaques in Aging and Age-Related Drusen,https://pmc.ncbi.nlm.nih.gov/articles/PMC7425688,GOLD,"\nD iseases of the macula, such as age-related macular degeneration (AMD) and diabetic macular edema, are leading causes of visual impairment in developed countries. 1 Animal models of macular conditions can further detail the mechanisms of their pathogenesis and reveal new insights into developing novel interventions. Nonhuman primates (NHPs) are a compelling animal model for studying macular diseases as they are the only mammals beside humans to possess a true macula. NHPs, such as rhesus ...","[{""attributes"":null,""end"":1340,""start"":1},{""attributes"":null,""end"":2554,""start"":1342},{""attributes"":null,""end"":4618,""start"":2580},{""attributes"":null,""end"":6008,""start"":4640},{""attributes"":null,""end"":7798,""start"":6010},{""attributes"":null,""end"":9037,""start"":7824},{""attributes"":null,""end"":10039,""start"":9078},{""attributes"":null,""end"":11377,""start"":10063},{""attributes"":null,""end"":12469,""start"":11391},{""attributes"":null,""end"":13219,""start"":12492},{""attributes"":null,""end"":13875,""start"":13257},{""att...","[{""attributes"":null,""end"":2563,""start"":2556},{""attributes"":null,""end"":2578,""start"":2565},{""attributes"":null,""end"":4638,""start"":4620},{""attributes"":null,""end"":7822,""start"":7800},{""attributes"":null,""end"":9076,""start"":9039},{""attributes"":null,""end"":10061,""start"":10041},{""attributes"":null,""end"":11389,""start"":11379},{""attributes"":null,""end"":12478,""start"":12471},{""attributes"":null,""end"":12490,""start"":12480},{""attributes"":null,""end"":13255,""start"":13221},{""attributes"":null,""end"":14316,""start"":14284}...",CCBYNCND,"Purpose To employ quantitative fundus autofluorescence (qAF) imaging in rhesus macaques to noninvasively assess retinal pigment epithelial (RPE) lipofuscin in nonhuman primates (NHPs) as a model of aging and age-related macular degeneration (AMD). Methods The qAF imaging was performed on eyes of 26 rhesus macaques (mean age 18.8 ± 8.2 years, range 4–27 years) with normal-appearing fundus or with age-related soft drusen using a confocal scanning laser ophthalmoscope with 488 nm excitation and...",2020,2020-07-01
1,268446036,<NA>,10.1007/s40670-024-02017-9,<NA>,Humanism Rounds: A Multifaceted “Back to Bedside” Initiative to Improve Meaning at Work for Internal Medicine Residents,https://pmc.ncbi.nlm.nih.gov/articles/PMC11180076,HYBRID,"\nIntroduction\n\nBurnout affects medical residents nationwide, leading to poor resident wellbeing, career dissatisfaction, and decreased quality of patient care [1,2].The rates of burnout among residents range from 27 to 75%, with high rates noted in obstetrics and gynecology (75%), internal medicine (63%), and general surgery (40%) with the lowest rate among family medicine residents (27%) [3].Research into burnout during residency has focused on a variety of contributing factors including...","[{""attributes"":null,""end"":1032,""start"":15},{""attributes"":null,""end"":1487,""start"":1034},{""attributes"":null,""end"":1960,""start"":1489},{""attributes"":null,""end"":2269,""start"":1985},{""attributes"":null,""end"":3087,""start"":2291},{""attributes"":null,""end"":3559,""start"":3118},{""attributes"":null,""end"":4961,""start"":3612},{""attributes"":null,""end"":5991,""start"":4980},{""attributes"":null,""end"":6547,""start"":5993},{""attributes"":null,""end"":7174,""start"":6571},{""attributes"":null,""end"":7594,""start"":7185},{""attributes""...","[{""attributes"":null,""end"":13,""start"":1},{""attributes"":null,""end"":1983,""start"":1962},{""attributes"":null,""end"":2289,""start"":2271}

,c
0,11609787


 :: "02_stg".stg_semanticscholar_combined_works queries finished | since_start: 13.0 minutes, 58.61 seconds | since_last: 10.29 seconds :: 


' :: "02_stg".stg_semanticscholar_combined_works queries finished | since_start: 13.0 minutes, 58.61 seconds | since_last: 10.29 seconds :: '

In [5]:
table_name = 'base_arxiv_metadata'
timelogger = utils.TimeLogger()
utils.create_table_from_sql_file(
    database_name = '02_stg',
    table_name = table_name,
    overwrite_strategy='overwrite', # options: fail, overwrite, ignore
    wait=True,
)
timelogger.log(f'"02_stg".{table_name} created')

utils.pd_set_options(cols=500)
display(wr.athena.read_sql_query(f"""SELECT * FROM "02_stg".{table_name} LIMIT 3 """, '02_stg'))
display(wr.athena.read_sql_query(f"""SELECT COUNT(*) AS c FROM "02_stg".{table_name} """, '02_stg'))
timelogger.log(f'"02_stg".{table_name} queries finished')

 :: :: TIMELOGGER STARTED :: | since_start: 0.00 seconds | since_last: 0.00 seconds :: 
Table 02_stg.base_arxiv_metadata already exists. Overwriting since overwrite_strategy=="overwrite".
Deleting table from Glue Catalog 02_stg base_arxiv_metadata
Deleting S3 objects from s3://sagemaker-research-methodology-extraction/01_data/02_stg/base_arxiv_metadata/
s3_parent_target_path:  s3://sagemaker-research-methodology-extraction/01_data/02_stg
 :: "02_stg".base_arxiv_metadata created | since_start: 15.44 seconds | since_last: 15.44 seconds :: 


,id_arxiv,id_doi,title,abstract,license
0,0903.1601,<NA>,Parabolic-Dish Solar Concentrators of Film on Foam,"Parabolic and spherical mirrors are constructed of aluminized PET polyester film on urethane foam. During construction, the chosen shape of the mirror is created by manipulating the elastic/plastic behavior of the film with air pressure. Foam is then applied to the film and, once hardened, air pressure is removed. At an f-number of 0.68, preliminary models have an optical angular spread of less than 0.25 degrees, a factor of 3.3 smaller than that for a perfectly spherical mirror. The possi...",ArXiv nonexclusive-distrib
1,0903.1604,10.3842/SIGMA.2009.029,Limits of Gaudin Systems: Classical and Quantum Cases,"We consider the XXX homogeneous Gaudin system with $N$ sites, both in classical and the quantum case. In particular we show that a suitable limiting procedure for letting the poles of its Lax matrix collide can be used to define new families of Liouville integrals (in the classical case) and new ""Gaudin"" algebras (in the quantum case). We will especially treat the case of total collisions, that gives rise to (a generalization of) the so called Bending flows of Kapovich and Millson. Some as...",CCBYNCSA
2,0903.1600,<NA>,Typically Real Harmonic Functions,We consider a class $\THO$ of typically real harmonic functions on the unit disk that contains the class of normalized analytic and typically real functions. We also obtain some partial results about the region of univalence for this class.,ArXiv nonexclusive-distrib


,c
0,2816721


 :: "02_stg".base_arxiv_metadata queries finished | since_start: 24.78 seconds | since_last: 9.35 seconds :: 


' :: "02_stg".base_arxiv_metadata queries finished | since_start: 24.78 seconds | since_last: 9.35 seconds :: '

In [6]:
table_name = 'stg_unified_works_01_joined_to_arxiv'
timelogger = utils.TimeLogger()
utils.create_table_from_sql_file(
    database_name = '02_stg',
    table_name = table_name,
    overwrite_strategy='overwrite', # options: fail, overwrite, ignore
    wait=True,
)
timelogger.log(f'"02_stg".{table_name} created')

utils.pd_set_options(cols=300)
display(wr.athena.read_sql_query(f"""SELECT * FROM "02_stg".{table_name} LIMIT 3 """, '02_stg'))
display(wr.athena.read_sql_query(f"""SELECT COUNT(*) AS c FROM "02_stg".{table_name} """, '02_stg'))
timelogger.log(f'"02_stg".{table_name} queries finished')

 :: :: TIMELOGGER STARTED :: | since_start: 0.00 seconds | since_last: 0.00 seconds :: 
Table 02_stg.stg_unified_works_01_joined_to_arxiv already exists. Overwriting since overwrite_strategy=="overwrite".
Deleting table from Glue Catalog 02_stg stg_unified_works_01_joined_to_arxiv
Deleting S3 objects from s3://sagemaker-research-methodology-extraction/01_data/02_stg/stg_unified_works_01_joined_to_arxiv/
s3_parent_target_path:  s3://sagemaker-research-methodology-extraction/01_data/02_stg
 :: "02_stg".stg_unified_works_01_joined_to_arxiv created | since_start: 11.0 minutes, 30.52 seconds | since_last: 11.0 minutes, 30.52 seconds :: 


,id_semanticscholar,id_mag,id_doi,id_arxiv,title,source_url,openaccess_status,content_text,annotations_paragraph,annotations_section_header,content_abstract,publication_year,publication_date,license,license_allows_derivative_reuse
0,17158598,1967400534,10.1103/physreve.68.026211,cond-mat/0206124,Fractal entropy of a chain of nonlinear oscillators.,https://arxiv.org/abs/cond-mat/0206124,GREEN,"\nINTRODUCTION\n\nA system composed of a large number of particles (generically) relaxes toward an equilibrium state that is independent of the details of the initial state. This is one of the fundamental hypotheses of statistical mechanics. However, from the point of view of Hamiltonian dynamic...","[{""attributes"":null,""end"":575,""start"":15},{""attributes"":null,""end"":1410,""start"":577},{""attributes"":null,""end"":1952,""start"":1412},{""attributes"":null,""end"":1970,""start"":1959},{""attributes"":null,""end"":2504,""start"":1972},{""attributes"":null,""end"":2510,""start"":2506},{""attributes"":null,""end"":2820,""star...","[{""attributes"":null,""end"":13,""start"":1},{""attributes"":null,""end"":1957,""start"":1954},{""attributes"":null,""end"":3641,""start"":3620},{""attributes"":null,""end"":7257,""start"":7233},{""attributes"":null,""end"":10659,""start"":10610},{""attributes"":null,""end"":14779,""start"":14764},{""attributes"":null,""end"":16435,""...",We study the time evolution of a chain of nonlinear oscillators. We focus on the fractal features of the spectral entropy and analyze its characteristic intermediate time scales as a function of the nonlinear coupling. A Brownian motion is recognized with an analytic power-law dependence of its ...,2002,2002-06-08,unknown-reusability,0
1,235731897,<NA>,<NA>,2107.01985,Pseudo-elliptic geometry of a class of Frobenius-manifolds&Maurer--Cartan structures,https://arxiv.org/abs/2107.01985,<NA>,"\nIntroduction\n\nThe notion of Frobenius manifolds (resp. F -manifolds) is the fruit of fifty years of remarkable interaction between topology and quantum physics. This relation involves the most advanced and sophisticated ideas on each side, and lead to Topological Quantum Field Theory. \n\nTh...","[{""attributes"":null,""end"":287,""start"":15},{""attributes"":null,""end"":650,""start"":289},{""attributes"":null,""end"":903,""start"":652},{""attributes"":null,""end"":1108,""start"":905},{""attributes"":null,""end"":1570,""start"":1110},{""attributes"":null,""end"":1929,""start"":1572},{""attributes"":null,""end"":2149,""start"":1...","[{""attributes"":null,""end"":13,""start"":1},{""attributes"":null,""end"":4008,""start"":3907},{""attributes"":null,""end"":5063,""start"":5045},{""attributes"":null,""end"":11672,""start"":11610},{""attributes"":{""n"":""2.""},""end"":12613,""start"":12577},{""attributes"":{""n"":""2.1.""},""end"":12838,""start"":12791},{""attributes"":nu...","The recently discovered fourth class of Frobenius manifolds by Combe–Manin in [11] opened and highlighted new geometric domains to explore. The guiding mantra of this article is to show the existence of hidden geometric aspects of the fourth Frobenius manifold, which turns out to be related to s...",2021,2021-07-05,ArXiv nonexclusive-distrib,0
2,20895502,2734548882,<NA>,1707.03678,Inverting normative city theories and computational urban models: Towards a coexistence with urban data streams,https://arxiv.org/abs/1707.03678,<NA>,"\nIntroduction: Model, theory, genealogy\n\nThe Oxford English Dictionary defines theory as a supposition or a system of ideas intended to explain something, especially one based on general principles independent of the thing to be explained, and a model as something such as a system that can be...","[{""attributes"":null,""end"":811,""start"":41},{""attributes"":null,""end"":3371,""start"":813},{""attributes"":null,""end"":4146,""start"":3373},{""attributes"":null,""end"":5335,""start"":4148},{""attributes"":null,""end"":6176,""start"":5373},{""attributes"":null,""end"":6873,""start"":

,c
0,11609787


 :: "02_stg".stg_unified_works_01_joined_to_arxiv queries finished | since_start: 11.0 minutes, 39.81 seconds | since_last: 9.28 seconds :: 


' :: "02_stg".stg_unified_works_01_joined_to_arxiv queries finished | since_start: 11.0 minutes, 39.81 seconds | since_last: 9.28 seconds :: '

In [ ]:
table_name = 'stg_unified_works_02_joined_to_openalex'
timelogger = utils.TimeLogger()
utils.create_table_from_sql_file(
    database_name = '02_stg',
    table_name = table_name,
    overwrite_strategy='overwrite', # options: fail, overwrite, ignore
    wait=True,
)
timelogger.log(f'"02_stg".{table_name} created')

utils.pd_set_options(cols=300)
display(wr.athena.read_sql_query(f"""SELECT * FROM "02_stg".{table_name} LIMIT 3 """, '02_stg'))
display(wr.athena.read_sql_query(f"""SELECT COUNT(*) AS c FROM "02_stg".{table_name} """, '02_stg'))
timelogger.log(f'"02_stg".{table_name} queries finished')

 :: :: TIMELOGGER STARTED :: | since_start: 0.00 seconds | since_last: 0.00 seconds :: 
Deleting S3 objects from s3://sagemaker-research-methodology-extraction/01_data/02_stg/stg_unified_works_02_joined_to_openalex/
s3_parent_target_path:  s3://sagemaker-research-methodology-extraction/01_data/02_stg


In [ ]:
utils.pd_set_options(cols=500)

wr.athena.read_sql_query("""
WITH
count_raw AS (
SELECT 1 AS i, '01_raw.semanticscholar_s2orc_v2' AS t, COUNT(*) AS c FROM "01_raw".semanticscholar_s2orc_v2
),
count_base AS (
SELECT 2 AS i, '02_stg.base_semanticscholar_s2orcv2' AS t, COUNT(*) AS c FROM "02_stg".base_semanticscholar_s2orcv2
),
count_stg_combined AS (
SELECT 2 AS i, '02_stg.stg_semanticscholar_combined_works' AS t, COUNT(*) AS c FROM "02_stg".stg_semanticscholar_combined_works
),
count_stg_joined_step01 AS (
SELECT 3 AS i, '02_stg.stg_unified_works_01_joined_to_arxiv' AS t, COUNT(*) AS c FROM "02_stg".stg_unified_works_01_joined_to_arxiv
),
count_stg_joined_step02 AS (
SELECT 3 AS i, '02_stg.stg_unified_works_02_joined_to_openalex' AS t, COUNT(*) AS c FROM "02_stg".stg_unified_works_02_joined_to_openalex
)
SELECT * FROM count_raw UNION ALL
SELECT * FROM count_base UNION ALL
SELECT * FROM count_stg_combined UNION ALL
SELECT * FROM count_stg_joined_step01 UNION ALL
SELECT * FROM count_stg_joined_step02

 """, '01_raw')